## Librerías

In [1]:
# qa_stress_test.ipynb

# --- 0. Configuración Inicial y Carga de Librerías ---
import pandas as pd
import numpy as np
import os
import json
import yaml
import time
from openai import OpenAI
# Importación de utils asumiendo que está en la misma carpeta 'notebooks/'
# Asegúrate de que utils.py contenga la clase TimerResult y el @contextmanager def timer(name): yield result
from utils import timer 

In [2]:
# Para asegurar que se muestren todas las columnas de Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## path

In [3]:
# >>>>> RUTA BASE PERSONALIZADA DEL PROYECTO (INTEGRACIÓN DE TU pk_) <<<<<
# Esta ruta es esencial para que el notebook encuentre los archivos
# generados por data_generator.py (data/, company_docs/, etc.)
pk_ = "C:/Users/Alber/OneDrive/Documentos/MADUREZ MLOPS/gerente-relacional-qa-test/"
# Puedes ajustar esta variable si la ubicación de tu proyecto cambia.
# >>>>> FIN DE RUTA BASE PERSONALIZADA <<<<<

## open AI

In [4]:
# --- 1. Cargar Credenciales ---
print("Cargando credenciales de OpenAI...")
try:
    # Usar pk_ para la ruta de credentials.json
    with open(os.path.join('credentials.json')) as f:
        config_env = json.load(f)
    api_key = config_env["openai_key"]
    client = OpenAI(api_key=api_key)
    print("Credenciales cargadas. Cliente OpenAI inicializado.")
except FileNotFoundError:
    print(f"ERROR: 'credentials.json' no encontrado en la ruta: {os.path.join(pk_, 'credentials.json')}. Asegúrate de crearlo en la raíz de tu proyecto ('{pk_}').")
    api_key = None
    client = None
except KeyError:
    print("ERROR: 'openai_key' no encontrada en 'credentials.json'.")
    api_key = None
    client = None


Cargando credenciales de OpenAI...
Credenciales cargadas. Cliente OpenAI inicializado.


## YAML Prompts

In [5]:
# --- 2. Cargar Prompts del Reporte Unificado ---
print("\nCargando prompts desde prompts.yml...")
# Usar pk_ para la ruta de prompts.yml
prompts_path = os.path.join('prompts.yml')
unified_report_prompts = {}
with timer("Carga de prompts y queries") as t_prompts: # Timer para esta etapa
    try:
        with open(prompts_path, 'r', encoding='utf-8') as file:
            all_prompts = yaml.safe_load(file)
        unified_report_prompts = {k: v for k, v in all_prompts.items() if k.startswith('unified_report_')}
        print(f"Prompts de reporte unificado cargados: {list(unified_report_prompts.keys())}")
    except FileNotFoundError:
        print(f"ERROR: '{prompts_path}' no encontrado.")
    except yaml.YAMLError as e:
        print(f"ERROR al parsear YAML: {e}")



Cargando prompts desde prompts.yml...
Prompts de reporte unificado cargados: ['unified_report_1', 'unified_report_2', 'unified_report_3', 'unified_report_4', 'unified_report_5']
[TIMER] Carga de prompts y queries: 0.046s


## Data

In [6]:
# --- 3. Cargar Datas Simuladas (10M Transacciones y 300K Financieros) y Mapeo de Empresas ---
print("\nCargando datas simuladas...")
transactions_df = pd.DataFrame()
financial_df = pd.DataFrame()
company_mapping = [] # Lista de diccionarios {original_name, sanitized_folder_name}

# Decide qué versión de datos cargar (LIGHT o FULL)
USE_LIGHT_DATA = True # Cambia a False para usar los datos grandes (10M/300K)

transactions_file = 'simulated_transactions.csv'
financial_file = 'simulated_financial_metrics.csv'

with timer("Carga de dataframes e historial") as t_data_load: # Timer para esta etapa
    try:
        # Usar pk_ para la ruta de los archivos de datos
        transactions_df = pd.read_csv(os.path.join(pk_, 'data', transactions_file))
        print(f"Data de transacciones cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(transactions_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{transactions_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    try:
        # Usar pk_ para la ruta de los archivos de datos
        financial_df = pd.read_csv(os.path.join(pk_, 'data', financial_file))
        print(f"Data financiera cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(financial_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{financial_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    # Cargar el mapeo de empresas
    try:
        # Usar pk_ para la ruta del mapeo de empresas
        with open(os.path.join(pk_, 'data', 'company_mapping.json'), 'r', encoding='utf-8') as f:
            company_mapping = json.load(f)
        print(f"Mapeo de {len(company_mapping)} empresas cargado.")
    except FileNotFoundError:
        print(f"ERROR: 'company_mapping.json' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")
    except json.JSONDecodeError:
        print("ERROR: Fallo al leer 'company_mapping.json'. Asegúrate de que es un JSON válido.")



Cargando datas simuladas...
Data de transacciones cargada (LIGHT). Registros: 2000
Data financiera cargada (LIGHT). Registros: 1000
Mapeo de 25 empresas cargado.
[TIMER] Carga de dataframes e historial: 0.026s


In [7]:
transactions_df.head(1)

,transaction_id,company_name,date,amount,type,description
0,0,Posada-Peña S.A.S.,2024-09-10,586644.59,DEBIT,Libero porro quam modi incidunt.


In [8]:
financial_df.head(1)

,financial_id,company_name,year,revenue,profit,liquidity_ratio,debt_equity_ratio,cash_flow
0,0,Gómez-Cardozo S.A.S.,2021,14103440.43,19772809.19,0.69,2.1,9111115.75


## FLOW

In [10]:
# --- 4. Función para Cargar Contenido de PDFs Simulados por Empresa (Tu versión adaptada) ---
def load_company_documents(sanitized_folder_name):
    """Carga el contenido textual simulado de los PDFs para una empresa usando su nombre de carpeta sanitizado."""
    doc_contents = {}
    # Usar pk_ para la ruta base de los documentos de las empresas
    base_path = os.path.join(pk_, 'company_docs', sanitized_folder_name)
    if not os.path.exists(base_path):
        print(f"Advertencia: Carpeta de documentos no encontrada para '{sanitized_folder_name}' en '{base_path}'")
        return doc_contents

    # Este timer imprime su tiempo, pero su métrica se capturará indirectamente
    # en "Creación de PineconeManagers (carga de documentos)" de run_unified_report_flow.
    with timer(f"Carga documentos para {sanitized_folder_name}") as t_doc_load:
        for doc_type in ['gestion', 'sectorial', 'financiero']:
            file_path = os.path.join(base_path, f'{doc_type}.txt')
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    doc_contents[doc_type] = f.read()
            else:
                doc_contents[doc_type] = "" # Vacío si no se encuentra el archivo
    return doc_contents

# --- 5. Lógica Principal de Generación de Reporte (Emulación del main.py) ---

def run_unified_report_flow(company_original_name, company_sanitized_folder_name, report_prompt_key, user_query=""):
    """
    Ejecuta un flujo simulado de generación de reporte unificado para una empresa.
    Mide tiempos y recolecta métricas.
    """
    # Asegúrate de que client, unified_report_prompts, company_mapping, transactions_df, financial_df
    # estén definidos globalmente o pasados como argumentos.
    # Si no, esta comprobación fallará.
    if 'client' not in globals() or client is None or \
       'unified_report_prompts' not in globals() or not unified_report_prompts or \
       'company_mapping' not in globals() or not company_mapping or \
       'transactions_df' not in globals() or transactions_df.empty or \
       'financial_df' not in globals() or financial_df.empty:
        print("ERROR: Variables globales (client, prompts, mapping, dataframes) no inicializadas. Abortando.")
        return {"status": "failed", "error": "Setup incomplete"}


    print(f"\n--- Ejecutando flujo para {company_original_name} ({report_prompt_key}) ---")
    metrics = {
        "status": "success",
        "total_execution_time": 0.0,
        "llm_input_tokens": 0,
        "llm_output_tokens": 0,
        "data_processed_tx_rows": 0,
        "data_processed_fin_rows": 0,
        "llm_api_latency": 0.0, # Latencia de la llamada al LLM aislada
        "timer_metrics": {} # Para guardar los tiempos de cada [TIMER]
    }
    
    # Timer principal para el flujo completo
    with timer("Tiempo total conversación") as total_timer_result: 
        # --- [TIMER] Normalización y decisión de flujo: (SIMULADO)
        with timer("Normalización y decisión de flujo") as t_norm_result:
            time.sleep(0.001) # Pequeña simulación de procesamiento
        # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
        print(f"DEBUG_NORM: t_norm_result object: {t_norm_result}")
        if t_norm_result is not None:
            print(f"DEBUG_NORM: t_norm_result.elapsed_time: {t_norm_result.elapsed_time}")
        else:
            print("DEBUG_NORM: t_norm_result es None.")
        # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
        metrics["timer_metrics"]["Normalización y decisión de flujo"] = t_norm_result.elapsed_time
        
        # --- [TIMER] Inicialización cliente e historial: (SIMULADO)
        with timer("Inicialización cliente e historial") as t_init_result:
            time.sleep(0.01) # Pequeña simulación
        # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
        print(f"DEBUG_INIT: t_init_result object: {t_init_result}")
        if t_init_result is not None:
            print(f"DEBUG_INIT: t_init_result.elapsed_time: {t_init_result.elapsed_time}")
        else:
            print("DEBUG_INIT: t_init_result es None.")
        # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
        metrics["timer_metrics"]["Inicialización cliente e historial"] = t_init_result.elapsed_time

        # --- [TIMER] Configuración y modelos: (SIMULADO)
        with timer("Configuración y modelos") as t_config_result:
            time.sleep(0.05) # Simulación de tiempo para configuración
        # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
        print(f"DEBUG_CONFIG: t_config_result object: {t_config_result}")
        if t_config_result is not None:
            print(f"DEBUG_CONFIG: t_config_result.elapsed_time: {t_config_result.elapsed_time}")
        else:
            print("DEBUG_CONFIG: t_config_result es None.")
        # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
        metrics["timer_metrics"]["Configuración y modelos"] = t_config_result.elapsed_time
        
        # --- [TIMER] Creación de PineconeManagers (incluye carga de docs simulada):
        with timer("Creación de PineconeManagers (carga de documentos)") as t_pinecone_init_result:
            company_docs = load_company_documents(company_sanitized_folder_name)
        # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
        print(f"DEBUG_PINECONE: t_pinecone_init_result object: {t_pinecone_init_result}")
        if t_pinecone_init_result is not None:
            print(f"DEBUG_PINECONE: t_pinecone_init_result.elapsed_time: {t_pinecone_init_result.elapsed_time}")
        else:
            print("DEBUG_PINECONE: t_pinecone_init_result es None.")
        # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
        metrics["timer_metrics"]["Creación de PineconeManagers (carga de documentos)"] = t_pinecone_init_result.elapsed_time
        
        # --- Preparar el Prompt Final para el LLM ---
        current_report_prompt_template = unified_report_prompts.get(report_prompt_key)
        if not current_report_prompt_template:
            print(f"ERROR: Prompt '{report_prompt_key}' no encontrado.")
            metrics["status"] = "failed"
            return metrics
        
        # --- [TIMER] Filtrado de dataframes por empresa (simulando query a BD) ---
        with timer("Filtrado de dataframes por empresa (simulando query a BD)") as t_filter_df_result:
            company_transactions_df = transactions_df[transactions_df['company_name'] == company_original_name].copy()
            company_financial_df = financial_df[financial_df['company_name'] == company_original_name].copy()
            metrics["data_processed_tx_rows"] = len(company_transactions_df)
            metrics["data_processed_fin_rows"] = len(company_financial_df)
        # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
        print(f"DEBUG_FILTER: t_filter_df_result object: {t_filter_df_result}")
        if t_filter_df_result is not None:
            print(f"DEBUG_FILTER: t_filter_df_result.elapsed_time: {t_filter_df_result.elapsed_time}")
        else:
            print("DEBUG_FILTER: t_filter_df_result es None.")
        # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
        metrics["timer_metrics"]["Filtrado de dataframes por empresa (simulando query a BD)"] = t_filter_df_result.elapsed_time

        # --- [TIMER] Búsqueda similitud reporte unificado (simulando Pinecone/RAG):
        with timer("Búsqueda similitud reporte unificado") as t_rag_search_result:
            context_from_docs = company_docs.get('gestion', '') + "\n\n" + \
                                company_docs.get('sectorial', '') + "\n\n" + \
                                company_docs.get('financiero', '')
            
            time.sleep(0.01 + len(context_from_docs) / 1000000.0)
            
        # Esta ya la teníamos, la dejo para consistencia
        print(f"DEBUG_RAG: Tipo de t_rag_search_result después del bloque timer: {type(t_rag_search_result)}")
        if t_rag_search_result is not None:
            print(f"DEBUG_RAG: Valor de t_rag_search_result.elapsed_time: {t_rag_search_result.elapsed_time}")
        else:
            print("DEBUG_RAG: t_rag_search_result es None.")
        metrics["timer_metrics"]["Búsqueda similitud reporte unificado"] = t_rag_search_result.elapsed_time

        # --- Rellenar los placeholders del prompt ---
        simulated_products_list = "Préstamos Comerciales, Créditos de Liquidez, Cuentas de Ahorro, CDT."
        simulated_sector = "Tecnología y Servicios Financieros" 

        df_desem_pag_cast_md_str = company_transactions_df.head(5).to_markdown(index=False)
        df_perfilador_md_str = company_financial_df.head(5).to_markdown(index=False)

        response_va_simulated = "Según Valora Analitik, la empresa ha invertido en IA para optimizar procesos bancarios."
        response_pp_simulated = "En Primera Página se destacó la expansión regional de la empresa en el último año."

        try:
            formatted_prompt = current_report_prompt_template.format(
                company_name=company_original_name,
                user_request=user_query if user_query else f"Genera un reporte unificado para {company_original_name} basado en los datos proporcionados.",
                management_report=company_docs.get('gestion', 'No disponible'),
                sector=simulated_sector,
                sector_report=company_docs.get('sectorial', 'No disponible'),
                financial_report=company_docs.get('financiero', 'No disponible'),
                response_va=response_va_simulated,
                response_pp=response_pp_simulated,
                df_desem_pag_cast_md=df_desem_pag_cast_md_str,
                df_perfilador_md=df_perfilador_md_str,
                products_list=simulated_products_list
            )
        except KeyError as e:
            print(f"ERROR: Placeholder '{e}' no encontrado en el prompt '{report_prompt_key}'. Revisa tu prompts.yml.")
            metrics["status"] = "failed"
            return metrics

        full_messages = [
            {"role": "system", "content": formatted_prompt},
            {"role": "user", "content": user_query if user_query else f"Genera el reporte unificado para {company_original_name}."}
        ]

        # --- [TIMER] Generación prompt + invocación LLM (reporte) ---
        llm_invocation_start_time = time.perf_counter()
        try:
            completion = client.chat.completions.create(
                model="gpt-4", # Asegúrate de tener acceso a este modelo
                temperature=0.0,
                messages=full_messages
            )
            metrics["llm_api_latency"] = time.perf_counter() - llm_invocation_start_time
            
            response_content = completion.choices[0].message.content
            metrics["llm_input_tokens"] = completion.usage.prompt_tokens
            metrics["llm_output_tokens"] = completion.usage.completion_tokens
            
            # Usar la misma métrica de latencia de API para el timer_metrics
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = metrics["llm_api_latency"] 
            
            print(f"Respuesta del LLM generada (primeros 200 chars): {response_content[:200]}...")
        except Exception as e:
            print(f"ERROR en invocación LLM: {e}")
            metrics["status"] = "failed"
            metrics["error"] = str(e)
            # Registrar el tiempo del intento incluso si falla
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = time.perf_counter() - llm_invocation_start_time 
            response_content = "ERROR"
    
    # Captura el tiempo total del contexto principal al salir del 'with timer'
    metrics["total_execution_time"] = total_timer_result.elapsed_time

    print(f"\n--- Métricas Finales para {company_original_name} ({report_prompt_key}) ---")
    print(f"Tiempo Total de Ejecución: {metrics['total_execution_time']:.3f}s")
    print(f"Latencia de Invocación LLM (aislada): {metrics['llm_api_latency']:.3f}s")
    print(f"Tokens de Entrada LLM: {metrics['llm_input_tokens']}")
    print(f"Tokens de Salida LLM: {metrics['llm_output_tokens']}")
    print(f"Volumen de Transacciones procesadas: {metrics['data_processed_tx_rows']} filas")
    print(f"Volumen de Financieros procesados: {metrics['data_processed_fin_rows']} filas")
    print(f"Estado del flujo: {metrics['status']}")
    print("Tiempos por subproceso:")
    for k, v in metrics["timer_metrics"].items():
        if v is not None: # Asegurar que el valor no sea None antes de formatear
            print(f"  - {k}: {v:.3f}s")
        else:
            print(f"  - {k}: N/A (tiempo no capturado)")

    return metrics

## Latencia

In [11]:
company_mapping

[{'original_name': 'Muñoz LLC S.A.S.',
  'sanitized_folder_name': 'empresa_1_Munoz_LLC_SAS'},
 {'original_name': 'Posada-Peña S.A.S.',
  'sanitized_folder_name': 'empresa_2_Posada-Pena_SAS'},
 {'original_name': 'González Inc S.A.S.',
  'sanitized_folder_name': 'empresa_3_Gonzalez_Inc_SAS'},
 {'original_name': 'Gil-Salazar S.A.S.',
  'sanitized_folder_name': 'empresa_4_Gil-Salazar_SAS'},
 {'original_name': 'Galvis, Ramírez and Angarita S.A.S.',
  'sanitized_folder_name': 'empresa_5_Galvis_Ramirez_and_Angarita_SAS'},
 {'original_name': 'Gómez, Sanabria and Pulido S.A.S.',
  'sanitized_folder_name': 'empresa_6_Gomez_Sanabria_and_Pulido_SAS'},
 {'original_name': 'Buitrago, García and García S.A.S.',
  'sanitized_folder_name': 'empresa_7_Buitrago_Garcia_and_Garcia_SAS'},
 {'original_name': 'Reyes-Zambrano S.A.S.',
  'sanitized_folder_name': 'empresa_8_Reyes-Zambrano_SAS'},
 {'original_name': 'Pacheco, Velandia and Ávila S.A.S.',
  'sanitized_folder_name': 'empresa_9_Pacheco_Velandia_and_Avi

In [12]:
unified_report_prompts

{'unified_report_1': 'Eres un analista experto en la elaboración de informes gerenciales para el área comercial del Banco de Bogotá. Tu objetivo, juanto con otros analistas, es generar un informe estratégico comercial (detallado) que busca profundizar relaciones con el cliente {company_name}. Tienes asignado uno de los puntos del informe general.\nPara lograr tu tarea dentro de la división (entre analistas) de objetivo, sigue esta estructura, asegurando calidad analítica y redacción clara. Importante evitar repetir datos entre secciones (especialmente cifras, conclusiones y diagnósticos):\n\n1. Descripción general de la empresa: a partir de toda la información suministrada realiza un resumen que incluya la actividad de la empresa {company_name}; sus principales productos y servicios (realiza una lista que contenga descripciones breves para cada elemento); quiénes son los clientes de la empresa {company_name}; información de los competidores directos de la empresa {company_name} en el m

In [13]:
# --- 6. Ejecutar Pruebas (Ejemplo) ---
# Primero, asegúrate de haber ejecutado data_generator.py (o _light.py)
# para que los archivos CSV, TXT y company_mapping.json existan.

if not company_mapping:
    print("No se pudo cargar el mapeo de empresas. Asegúrate de ejecutar el generador de datos primero.")
else:
    test_company_data = company_mapping[0] # Tomamos la primera empresa del mapeo para la prueba de ejemplo
    test_company_original_name = test_company_data["original_name"]
    test_company_sanitized_folder_name = test_company_data["sanitized_folder_name"]

    print(f"\nRealizando una prueba de ejemplo para la empresa: {test_company_original_name}")
    
    if 'unified_report_1' in unified_report_prompts:
        results = run_unified_report_flow(
            company_original_name=test_company_original_name,
            company_sanitized_folder_name=test_company_sanitized_folder_name,
            report_prompt_key='unified_report_1',
            user_query="Genera el resumen general de la empresa con los datos proporcionados."
        )
        print("\n--- Resultados Detallados de la Prueba de Ejemplo ---")
        print(json.dumps(results, indent=2))
    else:
        print("El prompt 'unified_report_1' no está disponible en prompts.yml. Por favor, revisa tus prompts.")


Realizando una prueba de ejemplo para la empresa: Muñoz LLC S.A.S.

--- Ejecutando flujo para Muñoz LLC S.A.S. (unified_report_1) ---
[TIMER] Normalización y decisión de flujo: 0.004s
DEBUG_NORM: t_norm_result object: <utils.TimerResult object at 0x0000019E08CD15D0>
DEBUG_NORM: t_norm_result.elapsed_time: 0.003963099996326491
[TIMER] Inicialización cliente e historial: 0.010s
DEBUG_INIT: t_init_result object: <utils.TimerResult object at 0x0000019E08CD1360>
DEBUG_INIT: t_init_result.elapsed_time: 0.01006000000052154
[TIMER] Configuración y modelos: 0.051s
DEBUG_CONFIG: t_config_result object: <utils.TimerResult object at 0x0000019E08CD3550>
DEBUG_CONFIG: t_config_result.elapsed_time: 0.051328399997146334
[TIMER] Carga documentos para empresa_1_Munoz_LLC_SAS: 0.003s
[TIMER] Creación de PineconeManagers (carga de documentos): 0.003s
DEBUG_PINECONE: t_pinecone_init_result object: <utils.TimerResult object at 0x0000019E08CD0A90>
DEBUG_PINECONE: t_pinecone_init_result.elapsed_time: 0.00311

In [14]:
# --- 7. Planificación de Pruebas de Estrés y Recopilación de Resultados ---

# Este código te permite iterar sobre un subconjunto de empresas y diferentes prompts,
# para recopilar métricas de rendimiento.

all_test_results = []
num_companies_to_test = min(3, len(company_mapping)) # Puedes ajustar cuántas empresas probar (ej: 5, 10, o len(company_mapping) para todas)

print(f"\nIniciando pruebas de estrés para {num_companies_to_test} empresas...")

for i in range(num_companies_to_test):
    company_data = company_mapping[i]
    company_original_name = company_data["original_name"]
    company_sanitized_folder_name = company_data["sanitized_folder_name"] # Esta variable ya contiene la carpeta simulada

    print(f"\n--- Ejecutando pruebas para la empresa: {company_original_name} ---")

    # Puedes listar aquí los prompts que quieres probar.
    # Por ejemplo, si tienes 'unified_report_1', 'unified_report_2', etc.
    for prompt_key in ['unified_report_1']: # Empieza con uno o añade más como 'unified_report_2'
        if prompt_key in unified_report_prompts:
            print(f"  > Con prompt: {prompt_key}")
            result = run_unified_report_flow(
                company_original_name=company_original_name,
                company_sanitized_folder_name=company_sanitized_folder_name,
                report_prompt_key=prompt_key,
                user_query=f"Genera el reporte de {prompt_key} para {company_original_name}."
            )
            all_test_results.append({
                "company_name": company_original_name,
                "prompt_key": prompt_key,
                "metrics": result
            })
        else:
            print(f"  > Advertencia: El prompt '{prompt_key}' no se encontró en unified_report_prompts. Saltando.")

# --- Análisis y Guardado de Resultados ---
# Después de que todas las pruebas se ejecuten, 'all_test_results' contendrá un diccionario
# con las métricas de cada corrida.

print("\n--- Todas las pruebas de estrés ejecutadas ---")
print("Puedes analizar la variable 'all_test_results' para ver los resultados consolidados.")

# Ejemplo de cómo podrías imprimir un resumen básico de los resultados:
# for res in all_test_results:
#     print(f"\nEmpresa: {res['company_name']}, Prompt: {res['prompt_key']}")
#     print(f"  Estado: {res['metrics']['status']}")
#     print(f"  Tiempo Total: {res['metrics']['total_execution_time']:.3f}s")
#     print(f"  Latencia LLM: {res['metrics']['llm_api_latency']:.3f}s")
#     print(f"  Tokens Entrada/Salida: {res['metrics']['llm_input_tokens']}/{res['metrics']['llm_output_tokens']}")


Iniciando pruebas de estrés para 3 empresas...

--- Ejecutando pruebas para la empresa: Muñoz LLC S.A.S. ---
  > Con prompt: unified_report_1

--- Ejecutando flujo para Muñoz LLC S.A.S. (unified_report_1) ---
[TIMER] Normalización y decisión de flujo: 0.004s
DEBUG_NORM: t_norm_result object: <utils.TimerResult object at 0x0000019E08CD0160>
DEBUG_NORM: t_norm_result.elapsed_time: 0.0038657000040984713
[TIMER] Inicialización cliente e historial: 0.022s
DEBUG_INIT: t_init_result object: <utils.TimerResult object at 0x0000019E08CD3D90>
DEBUG_INIT: t_init_result.elapsed_time: 0.022233600000618026
[TIMER] Configuración y modelos: 0.058s
DEBUG_CONFIG: t_config_result object: <utils.TimerResult object at 0x0000019E08CD1810>
DEBUG_CONFIG: t_config_result.elapsed_time: 0.05843240000103833
[TIMER] Carga documentos para empresa_1_Munoz_LLC_SAS: 0.002s
[TIMER] Creación de PineconeManagers (carga de documentos): 0.002s
DEBUG_PINECONE: t_pinecone_init_result object: <utils.TimerResult object at 0x00

In [15]:
all_test_results[0]

{'company_name': 'Muñoz LLC S.A.S.',
 'prompt_key': 'unified_report_1',
 'metrics': {'status': 'success',
  'total_execution_time': 32.339360099998885,
  'llm_input_tokens': 1899,
  'llm_output_tokens': 608,
  'data_processed_tx_rows': 84,
  'data_processed_fin_rows': 48,
  'llm_api_latency': 32.22015090000059,
  'timer_metrics': {'Normalización y decisión de flujo': 0.0038657000040984713,
   'Inicialización cliente e historial': 0.022233600000618026,
   'Configuración y modelos': 0.05843240000103833,
   'Creación de PineconeManagers (carga de documentos)': 0.0020148000039625913,
   'Filtrado de dataframes por empresa (simulando query a BD)': 0.002628599999297876,
   'Búsqueda similitud reporte unificado': 0.02283970000280533,
   'Generación prompt + invocación LLM (reporte)': 32.22015090000059}}}

In [16]:
all_test_results[1]

{'company_name': 'Posada-Peña S.A.S.',
 'prompt_key': 'unified_report_1',
 'metrics': {'status': 'success',
  'total_execution_time': 23.341757599999255,
  'llm_input_tokens': 1917,
  'llm_output_tokens': 443,
  'data_processed_tx_rows': 82,
  'data_processed_fin_rows': 33,
  'llm_api_latency': 23.184409100002085,
  'timer_metrics': {'Normalización y decisión de flujo': 0.0026087999940500595,
   'Inicialización cliente e historial': 0.02109059999929741,
   'Configuración y modelos': 0.053297899998142384,
   'Creación de PineconeManagers (carga de documentos)': 0.06227450000005774,
   'Filtrado de dataframes por empresa (simulando query a BD)': 0.002097099997627083,
   'Búsqueda similitud reporte unificado': 0.009872900001937523,
   'Generación prompt + invocación LLM (reporte)': 23.184409100002085}}}

In [17]:
all_test_results[2]

{'company_name': 'González Inc S.A.S.',
 'prompt_key': 'unified_report_1',
 'metrics': {'status': 'success',
  'total_execution_time': 29.822612500000105,
  'llm_input_tokens': 1882,
  'llm_output_tokens': 601,
  'data_processed_tx_rows': 78,
  'data_processed_fin_rows': 38,
  'llm_api_latency': 29.648700799996732,
  'timer_metrics': {'Normalización y decisión de flujo': 0.004215900000417605,
   'Inicialización cliente e historial': 0.01544080000167014,
   'Configuración y modelos': 0.05886129999998957,
   'Creación de PineconeManagers (carga de documentos)': 0.06717000000207918,
   'Filtrado de dataframes por empresa (simulando query a BD)': 0.0035006000034627505,
   'Búsqueda similitud reporte unificado': 0.017144099998404272,
   'Generación prompt + invocación LLM (reporte)': 29.648700799996732}}}

## Propuesta

In [57]:
# --- 0. Configuración Inicial y Carga de Librerías ---
import pandas as pd
import numpy as np
import os
import json
import yaml
import time
from openai import OpenAI
from contextlib import contextmanager # Necesario para el decorador @contextmanager

# --- Definición de Timer (si utils.py no está en el PATH o por simplicidad) ---
class TimerResult:
    def __init__(self):
        self.elapsed_time = 0.0
        self.start_time = None
        self.end_time = None

@contextmanager
def timer(name):
    result = TimerResult()
    result.start_time = time.perf_counter()
    print(f"[{name}] Iniciando...")
    try:
        yield result
    finally:
        result.end_time = time.perf_counter()
        result.elapsed_time = result.end_time - result.start_time
        print(f"[{name}] Completado en {result.elapsed_time:.3f} segundos.")

# >>>>> RUTA BASE PERSONALIZADA DEL PROYECTO (INTEGRACIÓN DE TU pk_) <<<<<
pk_ = "C:/Users/Alber/OneDrive/Documentos/MADUREZ MLOPS/gerente-relacional-qa-test/"
# Puedes ajustar esta variable si la ubicación de tu proyecto cambia.
# >>>>> FIN DE RUTA BASE PERSONALIZADA <<<<<

# --- 1. Cargar Credenciales ---
print("Cargando credenciales de OpenAI...")
try:
    # CORREGIDO: Asegurado el uso de pk_ para la ruta de credentials.json
    with open(os.path.join('credentials.json')) as f:
        config_env = json.load(f)
    api_key = config_env["openai_key"]
    client = OpenAI(api_key=api_key)
    print("Credenciales cargadas. Cliente OpenAI inicializado.")
except FileNotFoundError:
    print(f"ERROR: 'credentials.json' no encontrado en la ruta: {os.path.join(pk_, 'credentials.json')}. Asegúrate de crearlo en la raíz de tu proyecto ('{pk_}').")
    api_key = None
    client = None
except KeyError:
    print("ERROR: 'openai_key' no encontrada en 'credentials.json'.")
    api_key = None
    client = None

# --- 2. Cargar Prompts del Reporte Unificado ---
print("\nCargando prompts desde prompts2.yml...")
# CORREGIDO: Asegurado el uso de pk_ para la ruta de prompts2.yml
prompts_path = os.path.join('prompts2.yml')
unified_report_prompts = {}
with timer("Carga de prompts y queries") as t_prompts:
    try:
        with open(prompts_path, 'r', encoding='utf-8') as file:
            all_prompts = yaml.safe_load(file)
        # Filtra los prompts relevantes
        unified_report_prompts = {k: v for k, v in all_prompts.items() if k.startswith('unified_report_') or k.startswith('hybrid_dynamic_update_')}
        print(f"Prompts de reporte unificado cargados: {list(unified_report_prompts.keys())}")
    except FileNotFoundError:
        print(f"ERROR: '{prompts_path}' no encontrado.")
    except yaml.YAMLError as e:
        print(f"ERROR al parsear YAML: {e}")

# ### CAMBIO CRÍTICO PARA DEMO: DEFINICIÓN EXPLÍCITA DEL PROMPT HÍBRIDO ###
# Esta es la definición del prompt que el usuario describió: toma el reporte pre-generado y lo actualiza.
# Deberías copiar esto a tu prompts2.yml si lo apruebas.
unified_report_prompts['hybrid_dynamic_update_prompt'] = """
Eres un asistente experto en finanzas.
Has sido proporcionado con un reporte unificado pre-generado sobre la empresa {company_name}.
Tu tarea es analizar este 'REPORTE PRE-GENERADO' en conjunto con los 'NUEVOS DATOS ACTUALIZADOS'.
Basándote en este análisis, debes identificar y presentar de manera concisa:
1.  **El insight clave** más relevante derivado de los nuevos datos y su impacto en el reporte pre-generado.
2.  **Una oportunidad de negocio** prioritaria que surja de esta nueva información.
3.  **Una priorización** clara de esta oportunidad o de las acciones a tomar.
4.  **Una frase** concisa que capture la esencia de la actualización.

Tu respuesta debe contener **ÚNICAMENTE estos cuatro puntos**, estructurados claramente, sin reescribir el reporte completo.

--- REPORTE PRE-GENERADO ---
{pre_generated_report}
--- FIN REPORTE PRE-GENERADO ---

--- RESÚMENES ORIGINALES DE CONTEXTO ---
- Resumen de Gestión (Original): {management_report_summary}
- Resumen Sectorial (Original): {sector_summary}
- Resumen Financiero (Original): {financial_report_summary}
--- FIN RESÚMENES ORIGINALES DE CONTEXTO ---

--- NUEVOS DATOS ACTUALIZADOS DE BIGQUERY ---
Datos de Desempeño y Pagos (Nuevos):
{df_desem_pag_cast_md_new}
Datos Perfilador Financiero (Nuevos):
{df_perfilador_md_new}
--- FIN NUEVOS DATOS ACTUALIZADOS ---

Considerando la solicitud del usuario: "{user_query}"
(Recuerda: Tu respuesta debe contener **ÚNICAMENTE** el insight, oportunidad, priorización y frase).
"""
# ### FIN CAMBIO CRÍTICO PARA DEMO ###

# --- 3. Cargar Datas Simuladas (10M Transacciones y 300K Financieros) y Mapeo de Empresas ---
print("\nCargando datas simuladas...")
transactions_df = pd.DataFrame()
financial_df = pd.DataFrame()
company_mapping = [] # Lista de diccionarios {original_name, sanitized_folder_name}

# Decide qué versión de datos cargar (LIGHT o FULL)
USE_LIGHT_DATA = True # Cambia a False para usar los datos grandes (10M/300K)

transactions_file = 'simulated_transactions.csv'
financial_file = 'simulated_financial_metrics.csv'

with timer("Carga de dataframes e historial") as t_data_load:
    try:
        # Usar pk_ para la ruta de los archivos de datos
        transactions_df = pd.read_csv(os.path.join(pk_, 'data', transactions_file))
        print(f"Data de transacciones cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(transactions_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{transactions_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    try:
        # Usar pk_ para la ruta de los archivos de datos
        financial_df = pd.read_csv(os.path.join(pk_, 'data', financial_file))
        print(f"Data financiera cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(financial_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{financial_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    # Cargar el mapeo de empresas
    try:
        # Usar pk_ para la ruta del mapeo de empresas
        with open(os.path.join(pk_, 'data', 'company_mapping.json'), 'r', encoding='utf-8') as f:
            company_mapping = json.load(f)
        print(f"Mapeo de {len(company_mapping)} empresas cargado.")
    except FileNotFoundError:
        print(f"ERROR: 'company_mapping.json' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")
    except json.JSONDecodeError:
        print("ERROR: Fallo al leer 'company_mapping.json'. Asegúrate de que es un JSON válido.")

# --- 4. Función para Cargar Contenido de PDFs Simulados por Empresa (Tu versión adaptada) ---
def load_company_documents(sanitized_folder_name):
    """Carga el contenido textual simulado de los PDFs para una empresa usando su nombre de carpeta sanitizado."""
    doc_contents = {}
    # Usar pk_ para la ruta base de los documentos de las empresas
    base_path = os.path.join(pk_, 'company_docs', sanitized_folder_name)
    if not os.path.exists(base_path):
        print(f"Advertencia: Carpeta de documentos no encontrada para '{sanitized_folder_name}' en '{base_path}'")
        return doc_contents

    with timer(f"Carga documentos para {sanitized_folder_name}") as t_doc_load:
        for doc_type in ['gestion', 'sectorial', 'financiero']:
            file_path = os.path.join(base_path, f'{doc_type}.txt')
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    doc_contents[doc_type] = f.read()
            else:
                doc_contents[doc_type] = "" # Vacío si no se encuentra el archivo
    return doc_contents

# --- 5. Lógica Principal de Generación de Reporte (Emulación del main.py) ---
# Esta función es la que quieres reutilizar para medir el tiempo "completo"
def run_unified_report_flow(company_original_name, company_sanitized_folder_name, report_prompt_key, user_query=""):
    """
    Ejecuta un flujo simulado de generación de reporte unificado para una empresa.
    Mide tiempos y recolecta métricas.
    """
    # Asegúrate de que client, unified_report_prompts, company_mapping, transactions_df, financial_df
    # estén definidos globalmente o pasados como argumentos.
    if 'client' not in globals() or client is None or \
       'unified_report_prompts' not in globals() or not unified_report_prompts or \
       'company_mapping' not in globals() or not company_mapping or \
       'transactions_df' not in globals() or transactions_df.empty or \
       'financial_df' not in globals() or financial_df.empty:
        print("ERROR: Variables globales (client, prompts, mapping, dataframes) no inicializadas. Abortando.")
        return {"status": "failed", "error": "Setup incomplete", "generated_content": "ERROR"}, "ERROR"

    print(f"\n--- Ejecutando flujo ORIGINAL para {company_original_name} ({report_prompt_key}) ---")
    metrics = {
        "status": "success",
        "total_execution_time": 0.0,
        "llm_input_tokens": 0,
        "llm_output_tokens": 0,
        "data_processed_tx_rows": 0,
        "data_processed_fin_rows": 0,
        "llm_api_latency": 0.0, # Latencia de la llamada al LLM aislada
        "timer_metrics": {} # Para guardar los tiempos de cada [TIMER]
    }
    generated_content = "" # Inicializar para asegurar que siempre se retorne

    with timer("Tiempo total conversación - ORIGINAL") as total_timer_result:
        with timer("Normalización y decisión de flujo") as t_norm_result:
            # time.sleep(0.001) # ### ELIMINADO time.sleep() ###
            pass
        metrics["timer_metrics"]["Normalización y decisión de flujo"] = t_norm_result.elapsed_time
        
        with timer("Inicialización cliente e historial") as t_init_result:
            # time.sleep(0.01) # ### ELIMINADO time.sleep() ###
            pass
        metrics["timer_metrics"]["Inicialización cliente e historial"] = t_init_result.elapsed_time

        with timer("Configuración y modelos") as t_config_result:
            # time.sleep(0.05) # ### ELIMINADO time.sleep() ###
            pass
        metrics["timer_metrics"]["Configuración y modelos"] = t_config_result.elapsed_time
        
        with timer("Creación de PineconeManagers (carga de documentos)") as t_pinecone_init_result:
            company_docs = load_company_documents(company_sanitized_folder_name)
        metrics["timer_metrics"]["Creación de PineconeManagers (carga de documentos)"] = t_pinecone_init_result.elapsed_time
        
        current_report_prompt_template = unified_report_prompts.get(report_prompt_key)
        if not current_report_prompt_template:
            print(f"ERROR: Prompt '{report_prompt_key}' no encontrado.")
            metrics["status"] = "failed"
            metrics["error"] = f"Prompt '{report_prompt_key}' no encontrado."
            return metrics, generated_content # Retornar también el contenido, incluso si es error
        
        with timer("Filtrado de dataframes por empresa (simulando query a BD)") as t_filter_df_result:
            company_transactions_df = transactions_df[transactions_df['company_name'] == company_original_name].copy()
            company_financial_df = financial_df[financial_df['company_name'] == company_original_name].copy()
            metrics["data_processed_tx_rows"] = len(company_transactions_df)
            metrics["data_processed_fin_rows"] = len(company_financial_df)
        metrics["timer_metrics"]["Filtrado de dataframes por empresa (simulando query a BD)"] = t_filter_df_result.elapsed_time

        with timer("Búsqueda similitud reporte unificado") as t_rag_search_result:
            context_from_docs = company_docs.get('gestion', '') + "\n\n" + \
                                company_docs.get('sectorial', '') + "\n\n" + \
                                company_docs.get('financiero', '')
            # time.sleep(0.01 + len(context_from_docs) / 1000000.0) # Simulación de RAG ### ELIMINADO time.sleep() ###
            pass
        metrics["timer_metrics"]["Búsqueda similitud reporte unificado"] = t_rag_search_result.elapsed_time

        # Rellenar los placeholders del prompt
        simulated_products_list = "Préstamos Comerciales, Créditos de Liquidez, Cuentas de Ahorro, CDT."
        simulated_sector = "Tecnología y Servicios Financieros" 

        df_desem_pag_cast_md_str = company_transactions_df.head(5).to_markdown(index=False)
        df_perfilador_md_str = company_financial_df.head(5).to_markdown(index=False)

        response_va_simulated = "Según Valora Analitik, la empresa ha invertido en IA para optimizar procesos bancarios."
        response_pp_simulated = "En Primera Página se destacó la expansión regional de la empresa en el último año."

        try:
            formatted_prompt = current_report_prompt_template.format(
                company_name=company_original_name,
                user_request=user_query if user_query else f"Genera un reporte unificado para {company_original_name} basado en los datos proporcionados.",
                management_report=company_docs.get('gestion', 'No disponible'),
                sector=simulated_sector,
                sector_report=company_docs.get('sectorial', 'No disponible'),
                financial_report=company_docs.get('financiero', 'No disponible'),
                response_va=response_va_simulated,
                response_pp=response_pp_simulated,
                df_desem_pag_cast_md=df_desem_pag_cast_md_str,
                df_perfilador_md=df_perfilador_md_str,
                products_list=simulated_products_list
            )
        except KeyError as e:
            print(f"ERROR: Placeholder '{e}' no encontrado en el prompt '{report_prompt_key}'. Revisa tu prompts.yml.")
            metrics["status"] = "failed"
            metrics["error"] = f"Placeholder '{e}' no encontrado."
            return metrics, generated_content # Retornar también el contenido, incluso si es error

        full_messages = [
            {"role": "system", "content": formatted_prompt},
            {"role": "user", "content": user_query if user_query else f"Genera el reporte unificado para {company_original_name}."}
        ]

        llm_invocation_start_time = time.perf_counter()
        try:
            completion = client.chat.completions.create(
                model="gpt-4", # Usamos gpt-4
                temperature=0.0,
                messages=full_messages
            )
            metrics["llm_api_latency"] = time.perf_counter() - llm_invocation_start_time
            
            generated_content = completion.choices[0].message.content
            metrics["llm_input_tokens"] = completion.usage.prompt_tokens
            metrics["llm_output_tokens"] = completion.usage.completion_tokens
            
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = metrics["llm_api_latency"] 
            
            print(f"Respuesta del LLM generada (primeros 200 chars): {generated_content[:200]}...")
        except Exception as e:
            print(f"ERROR en invocación LLM: {e}")
            metrics["status"] = "failed"
            metrics["error"] = str(e)
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = time.perf_counter() - llm_invocation_start_time 
            generated_content = "ERROR" # Asegurar que 'generated_content' tenga un valor
    
    metrics["total_execution_time"] = total_timer_result.elapsed_time

    print(f"\n--- Métricas Finales para {company_original_name} ({report_prompt_key}) ---")
    print(f"Tiempo Total de Ejecución: {metrics['total_execution_time']:.3f}s")
    print(f"Latencia de Invocación LLM (aislada): {metrics['llm_api_latency']:.3f}s")
    print(f"Tokens de Entrada LLM: {metrics['llm_input_tokens']}")
    print(f"Tokens de Salida LLM: {metrics['llm_output_tokens']}")
    print(f"Volumen de Transacciones procesadas: {metrics['data_processed_tx_rows']} filas")
    print(f"Volumen de Financieros procesados: {metrics['data_processed_fin_rows']} filas")
    print(f"Estado del flujo: {metrics['status']}")
    print("Tiempos por subproceso:")
    for k, v in metrics["timer_metrics"].items():
        if v is not None:
            print(f"   - {k}: {v:.3f}s")
        else:
            print(f"   - {k}: N/A (tiempo no capturado)")

    return metrics, generated_content # Ahora devuelve métricas y contenido generado

# --- NUEVAS FUNCIONES Y SECCIÓN DE MEDICIONES (AGREGADO) ---

# Mock de función para cargar "nuevos datos" de BigQuery.
# Tu código original la llamaba, así que la definimos para evitar errores.
# DEBES REEMPLAZAR ESTA LÓGICA SI TU ESCENARIO HÍBRIDO CARGA DATOS DIFERENTES.
def load_simulated_bigquery_data(simulation_version="default"):
    """
    Simula la carga de datos de BigQuery.
    En un escenario real, esto haría una consulta a tu BQ.
    """
    print(f"  Simulando carga de BigQuery para versión: {simulation_version}...")
    # time.sleep(0.1) # Simula una pequeña latencia ### ELIMINADO time.sleep() ###
    if simulation_version == "new_dynamic":
        # Datos ligeramente diferentes para simular "nuevos" datos
        tx_data = {'company_name': ['CompanyA', 'CompanyA'], 'value': [105, 205]}
        fin_data = {'company_name': ['CompanyA'], 'metric': [55.5]}
    else:
        # Usa los datos cargados globalmente como "default"
        tx_data = transactions_df.head(2).to_dict('list') if not transactions_df.empty else {'company_name': [], 'value': []}
        fin_data = financial_df.head(1).to_dict('list') if not financial_df.empty else {'company_name': [], 'metric': []}
        
    return pd.DataFrame(tx_data), pd.DataFrame(fin_data)

# Función general para la invocación al LLM (para el flujo híbrido)
# Asegura que capture tokens y latencia de manera similar a run_unified_report_flow
def get_llm_response(prompt_messages, model="gpt-4", temperature=0.5): 
    """
    Realiza la llamada al LLM y captura métricas.
    'prompt_messages' debe ser una lista de diccionarios como [{'role': 'system', 'content': '...'}, {'role': 'user', 'content': '...'}]
    """
    if client is None:
        print("ERROR: Cliente OpenAI no inicializado en get_llm_response. No se puede invocar LLM.")
        return "", {"status": "failed", "error": "LLM client not initialized", "llm_input_tokens": 0, "llm_output_tokens": 0, "llm_api_latency": 0.0}

    llm_invocation_start_time = time.perf_counter()
    response_content = ""
    input_tokens = 0
    output_tokens = 0
    status = "success"
    error_message = None

    try:
        completion = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=prompt_messages
        )
        response_content = completion.choices[0].message.content
        input_tokens = completion.usage.prompt_tokens
        output_tokens = completion.usage.completion_tokens
    except Exception as e:
        print(f"ERROR en get_llm_response: {e}")
        status = "failed"
        error_message = str(e)
        response_content = "ERROR EN LLM HÍBRIDO" # Aseguramos un valor de retorno
        
    llm_api_latency = time.perf_counter() - llm_invocation_start_time
    
    metrics = {
        "status": status,
        "error": error_message,
        "llm_input_tokens": input_tokens,
        "llm_output_tokens": output_tokens,
        "llm_api_latency": llm_api_latency,
    }
    return response_content, metrics

# --- Función para el Flujo Híbrido Completo con Métricas ---
def run_hybrid_update_flow(company_original_name, company_sanitized_folder_name, user_query, pre_generated_report_content=""):
    """
    Ejecuta el flujo de actualización híbrida y mide sus métricas.
    """
    print(f"\n--- Ejecutando flujo HÍBRIDO para {company_original_name} con consulta: '{user_query}' ---")
    
    metrics = {
        "status": "success",
        "total_execution_time": 0.0,
        "llm_input_tokens": 0,
        "llm_output_tokens": 0,
        "data_processed_tx_rows": 0,
        "data_processed_fin_rows": 0,
        "llm_api_latency": 0.0,
        "timer_metrics": {}
    }
    generated_content = ""

    with timer("Tiempo total conversación - HÍBRIDA") as total_timer_result:
        # Cargar los documentos estáticos para los resúmenes de contexto originales
        with timer("Carga de documentos estáticos para contexto (Híbrido)") as t_static_docs:
            company_docs = load_company_documents(company_sanitized_folder_name)
            # Volvemos a la lógica original de tomar los 200 primeros caracteres o el contenido completo si es corto
            management_summary = company_docs.get('gestion', '')[:200] if len(company_docs.get('gestion', '')) > 200 else company_docs.get('gestion', '')
            sector_summary = company_docs.get('sectorial', '')[:200] if len(company_docs.get('sectorial', '')) > 200 else company_docs.get('sectorial', '')
            financial_summary = company_docs.get('financiero', '')[:200] if len(company_docs.get('financiero', '')) > 200 else company_docs.get('financiero', '')
        metrics["timer_metrics"]["Carga de documentos estáticos para contexto (Híbrido)"] = t_static_docs.elapsed_time
        

        # Simular la llegada de NUEVOS datos de BigQuery (valores cambiados)
        with timer("Consulta a BigQuery (simulada, nuevos datos para Híbrido)") as t_bigquery_fetch:
            transactions_df_new_data, financial_df_new_data = load_simulated_bigquery_data(simulation_version="new_dynamic")
            metrics["data_processed_tx_rows"] = len(transactions_df_new_data)
            metrics["data_processed_fin_rows"] = len(financial_df_new_data)
        metrics["timer_metrics"]["Consulta a BigQuery (simulada, nuevos datos para Híbrido)"] = t_bigquery_fetch.elapsed_time

        df_desem_pag_cast_md_new_str = ""
        if not transactions_df_new_data.empty:
            df_desem_pag_cast_md_new_str = transactions_df_new_data.to_markdown(index=False) 
            
        df_perfilador_md_new_str = ""
        if not financial_df_new_data.empty:
            df_perfilador_md_new_str = financial_df_new_data.to_markdown(index=False) 

        # Construir el PROMPT OPTIMIZADO para el LLM
        optimized_prompt_template = unified_report_prompts.get('hybrid_dynamic_update_prompt')

        if not optimized_prompt_template:
            print("ERROR: El prompt 'hybrid_dynamic_update_prompt' no fue encontrado.")
            metrics["status"] = "failed"
            metrics["error"] = "Prompt 'hybrid_dynamic_update_prompt' no encontrado."
            return metrics, generated_content
        
        try:
            formatted_hybrid_prompt_text = optimized_prompt_template.format(
                company_name=company_original_name,
                pre_generated_report=pre_generated_report_content, # ¡Aquí pasamos el reporte completo pre-generado!
                management_report_summary=management_summary, # Ahora son de los docs, truncados
                sector_summary=sector_summary,                 # Ahora son de los docs, truncados
                financial_report_summary=financial_summary,   # Ahora son de los docs, truncados
                df_desem_pag_cast_md_new=df_desem_pag_cast_md_new_str,
                df_perfilador_md_new=df_perfilador_md_new_str,
                user_query=user_query
            )
        except KeyError as e:
            print(f"ERROR: Placeholder '{e}' no encontrado en el prompt optimizado. Revisa tu prompts.yml.")
            metrics["status"] = "failed"
            metrics["error"] = f"Placeholder '{e}' no encontrado en prompt híbrido."
            return metrics, generated_content

        # La llamada al LLM
        hybrid_messages = [
            {"role": "system", "content": formatted_hybrid_prompt_text},
            {"role": "user", "content": user_query}
        ]
        
        with timer("Invocación LLM (Actualización Dinámica Híbrida)") as t_llm_hybrid:
            generated_content, llm_metrics = get_llm_response(hybrid_messages, model="gpt-4", temperature=0.5) 

        metrics["llm_api_latency"] = llm_metrics["llm_api_latency"]
        metrics["llm_input_tokens"] = llm_metrics["llm_input_tokens"]
        metrics["llm_output_tokens"] = llm_metrics["llm_output_tokens"]
        metrics["status"] = llm_metrics["status"]
        if llm_metrics["error"]:
            metrics["error"] = llm_metrics["error"]

        metrics["timer_metrics"]["Invocación LLM (Actualización Dinámica Híbrida)"] = t_llm_hybrid.elapsed_time

        print(f"Respuesta del LLM híbrida (primeros 200 chars): {generated_content[:200]}...")
        # NOTA: La vista previa puede ser corta porque ahora el output del LLM está diseñado para ser conciso.

    metrics["total_execution_time"] = total_timer_result.elapsed_time

    print(f"\n--- Métricas Finales para {company_original_name} (Flujo Híbrido) ---")
    print(f"Tiempo Total de Ejecución: {metrics['total_execution_time']:.3f}s")
    print(f"Latencia de Invocación LLM (aislada): {metrics['llm_api_latency']:.3f}s")
    print(f"Tokens de Entrada LLM: {metrics['llm_input_tokens']}")
    print(f"Tokens de Salida LLM: {metrics['llm_output_tokens']}")
    print(f"Volumen de Transacciones procesadas: {metrics['data_processed_tx_rows']} filas")
    print(f"Volumen de Financieros procesados: {metrics['data_processed_fin_rows']} filas")
    print(f"Estado del flujo: {metrics['status']}")
    print("Tiempos por subproceso:")
    for k, v in metrics["timer_metrics"].items():
        if v is not None:
            print(f"   - {k}: {v:.3f}s")
        else:
            print(f"   - {k}: N/A (tiempo no capturado)")

    return metrics, generated_content


# --- SECCIÓN DE PRUEBAS DE ESTRÉS REAL ---

all_test_results = []
# Puedes ajustar cuántas empresas probar (ej: 1, 3, len(company_mapping) para todas)
num_companies_to_test = min(len(company_mapping), 1) # Probamos solo 1 empresa para no consumir muchos tokens

print(f"\n{'='*50}\nINICIANDO PRUEBAS DE ESTRÉS PARA {num_companies_to_test} EMPRESAS\n{'='*50}")

if not client:
    print("El cliente OpenAI no está inicializado. Asegúrate de que las credenciales son correctas. Abortando pruebas de estrés.")
else:
    for i in range(num_companies_to_test):
        company_data = company_mapping[i]
        company_original_name = company_data["original_name"]
        company_sanitized_folder_name = company_data["sanitized_folder_name"]

        print(f"\n{'*'*60}\nPROBANDO EMPRESA: {company_original_name}\n{'*'*60}")

        # --- PRUEBA 1: Generación del Reporte Unificado Inicial (Flujo Completo Original) ---
        print("\n[FLUJO ORIGINAL] Iniciando generación de reporte unificado...")
        initial_report_prompt_key = 'unified_report_1' # O el prompt que uses para el reporte inicial completo
        user_query_initial = f"Genera un reporte unificado de oportunidades de negocio para {company_original_name}."

        if initial_report_prompt_key in unified_report_prompts:
            initial_report_metrics, generated_initial_content = run_unified_report_flow(
                company_original_name=company_original_name,
                company_sanitized_folder_name=company_sanitized_folder_name,
                report_prompt_key=initial_report_prompt_key,
                user_query=user_query_initial
            )
            all_test_results.append({
                "flow_type": "initial_unified_report",
                "company_name": company_original_name,
                "prompt_key": initial_report_prompt_key,
                "user_query": user_query_initial,
                "metrics": initial_report_metrics,
                "generated_content_preview": generated_initial_content[:200] + "..." if generated_initial_content else ""
            })
            print(f"Resultado Flujo Original: Estado={initial_report_metrics['status']}, Tiempo={initial_report_metrics['total_execution_time']:.3f}s")
        else:
            print(f"Advertencia: Prompt '{initial_report_prompt_key}' no encontrado. Saltando prueba de reporte unificado.")
            generated_initial_content = "N/A" # Para que el flujo híbrido no falle si el inicial se salta

        # --- PRUEBA 2: Actualización Híbrida (Pre-generado + Actualización Dinámica) ---
        if 'hybrid_dynamic_update_prompt' in unified_report_prompts:
            print("\n[FLUJO HÍBRIDO] Iniciando actualización dinámica de reporte...")
            user_query_hybrid = f"Genera el insight clave, oportunidad, priorización y frase sobre los nuevos datos de {company_original_name}."
            
            hybrid_metrics, generated_hybrid_content = run_hybrid_update_flow(
                company_original_name=company_original_name,
                company_sanitized_folder_name=company_sanitized_folder_name,
                user_query=user_query_hybrid,
                pre_generated_report_content=generated_initial_content # Pasamos el contenido generado por el flujo inicial
            )
            all_test_results.append({
                "flow_type": "hybrid_dynamic_update",
                "company_name": company_original_name,
                "prompt_key": 'hybrid_dynamic_update_prompt',
                "user_query": user_query_hybrid,
                "metrics": hybrid_metrics,
                "generated_content_preview": generated_hybrid_content[:200] + "..." if generated_hybrid_content else ""
            })
            print(f"Resultado Flujo Híbrido: Estado={hybrid_metrics['status']}, Tiempo={hybrid_metrics['total_execution_time']:.3f}s")
        else:
            print(f"Advertencia: Prompt 'hybrid_dynamic_update_prompt' no encontrado. Saltando prueba de flujo híbrido.")


# --- Análisis de Resultados Globales ---
print(f"\n{'='*50}\nRESUMEN DE RESULTADOS DE PRUEBAS DE ESTRÉS\n{'='*50}")

if all_test_results:
    df_results = pd.DataFrame(all_test_results)
    
    # Resumen general
    print("\nResumen por tipo de flujo:")
    summary_by_flow = df_results.groupby('flow_type')['metrics'].apply(lambda x: pd.DataFrame(list(x)).mean(numeric_only=True)).reset_index()
    print(summary_by_flow.round(3).to_markdown(index=False))

    print("\nResultados detallados por empresa y flujo:")
    for result in all_test_results:
        print(f"\n--- {result['flow_type'].replace('_', ' ').title()} para {result['company_name']} ---")
        print(f"  Estado: {result['metrics']['status']}")
        print(f"  Tiempo Total: {result['metrics']['total_execution_time']:.3f}s")
        print(f"  Latencia LLM (aislada): {result['metrics']['llm_api_latency']:.3f}s")
        print(f"  Tokens In/Out: {result['metrics']['llm_input_tokens']}/{result['metrics']['llm_output_tokens']}")
        print(f"  Filas TX/FIN: {result['metrics']['data_processed_tx_rows']}/{result['metrics']['data_processed_fin_rows']}")
        if 'user_query' in result:
            print(f"  Consulta: {result['user_query']}")
        # Puedes añadir más detalles si es necesario
else:
    print("No se generaron resultados de pruebas. Revisa los mensajes de error anteriores.")

Cargando credenciales de OpenAI...
Credenciales cargadas. Cliente OpenAI inicializado.

Cargando prompts desde prompts2.yml...
[Carga de prompts y queries] Iniciando...
Prompts de reporte unificado cargados: ['unified_report_1', 'unified_report_2', 'unified_report_3', 'unified_report_4', 'unified_report_5', 'hybrid_dynamic_update_prompt']
[Carga de prompts y queries] Completado en 0.046 segundos.

Cargando datas simuladas...
[Carga de dataframes e historial] Iniciando...
Data de transacciones cargada (LIGHT). Registros: 2000
Data financiera cargada (LIGHT). Registros: 1000
Mapeo de 25 empresas cargado.
[Carga de dataframes e historial] Completado en 0.027 segundos.

INICIANDO PRUEBAS DE ESTRÉS PARA 1 EMPRESAS

************************************************************
PROBANDO EMPRESA: Muñoz LLC S.A.S.
************************************************************

[FLUJO ORIGINAL] Iniciando generación de reporte unificado...

--- Ejecutando flujo ORIGINAL para Muñoz LLC S.A.S. (unifie